## Welcome to the PKevo Tutorial Notebook
PKevo enables you to run Grammatical Evolution (short GE) simulations, during which polyketides are evolved and evaluated using a specified fitness function. Its primary goal is to generate de-novo polyketides which are best suited for docking against a specified pharmacophore. In this tutorial we will explore how PKevo works and achieves this goal by following a simplified toy example.

### About the structure of this tutorial
This tutorial is structured into steps, which highlight the main tasks (in chronological order) that PKevo is performing. Each step in further devided into:   
- a short textual explanation 
- a code block performing the described step
- an optional second code block, visualizing the performed changes in the prior code block (Note: You don't need to execute those code block unless you're interested in seeing what is going on "under the hood" of PKevo.) 

### Step 0: Preparations
Before we begin, let's import all the necessary components, which we will be using later on.

In [1]:
from rdkit.Chem import Descriptors
# In order to get access to PKevo from within the Jupyter Notebook, we need to adapt the path
import os
import sys
sys.path.insert(1, os.path.join(sys.path[0], '..')) # Adds the parent directory (which contains all PKevo packages) to the path

import random


# Initialize FileStorage class and check if logs folder exists
from config.file_storage_singleton import file_storage

if not os.path.exists(file_storage.logs_dir):
    file_storage.create_logs_folder()

from grammatical_evolution.model._individual import _Individual
from grammatical_evolution.model._grammar import _Grammar
from grammatical_evolution.model._molecule import Molecule
from grammatical_evolution.ge_utils.fitness_functions import FitnessFunctions
from grammatical_evolution.ge_utils.selection_strategies import tournament_selection
from grammatical_evolution.ge_utils.replacement_strategies import generational_replacement
from grammatical_evolution.ge_utils.crossovers import onepoint_crossover
from grammatical_evolution.ge_utils import mutations

from chemistry.chemistry import Chemistry

from config.pkevo_config import EvolutionConfig as evo_config
from config.pkevo_config import SearchConfig as search_config

### Step 1: Defining the environment
For a simulation we need to adjust our configuration. To configure the behaviour of the Evolutionary algorithm (a class called `EvolutionConfig`), you can generally rely on the default settings, but feel free to tweak the parameters as you like in order to further optimize your results. In this tutorial we will edit some of these parameters, but that is rather only to ensure that the simulation will be completed as quickly as possible, instead of focusing on getting good end results.

What you will almost always have to edit for your simulation are the parameters for the Search configuration (`SearchConfig` class), where your define your target, towards which the GE algorithm will try to evolve.

**Note:** You can find the aforementioned configuration file under: `/pkevo/config/pkevo_config.py`

In [2]:
# Note: Not all of these parameters will be used directly, however if one would be to run PKevo in a "Tutorial-like style"
# (something for which I can't think of a good reason other than pure boredom), this is how the settings would look like

# Ensure we have a rather small size of simpler individuals in a generation, as well as only two iterations overall
evo_config.POPULATION_SIZE = 4
evo_config.GENERATION_SIZE = 4
evo_config.GENERATIONS = 2
evo_config.GENOME_LENGTH = 40
evo_config.CODON_SIZE = 30
evo_config.MUTATION_PROBABILITY = 0.2
evo_config.CROSSOVER_PROBABILITY = 0.99
evo_config.SELECTION_STRATEGY = 'tournament_selection'
evo_config.REPLACEMENT_STRATEGY = 'generational_replacement'
evo_config.CROSSOVER_OPERATOR = 'onepoint_crossover'
# Set the target (since we want to use the Pharmacophore fitness function in this tutorial, we need to specify the pharmacophore file)
search_config.TARGET_PHARMACOPHORE_FILE = 'gdm_pharmacophore_relaxed.pmz'

### Step 2: Initializing the grammar for the Grammatical Evolution
As the name already suggests, for a Grammatical Evolution, we need to define and use a grammar. A grammar is a set of production rules, which the algorithm will later use to transform the genome of an individual into its expressed phenotype. In general GE a phenotype can be almost anything, but in PKevo a phenotype is the polyketide synthase (in short PKS) as a string of its domains and used molecules. 
- **Why is this important?** Think of the grammar as the "recipe book" for your polyketide synthesis. It outlines the rules and instructions that the GE algorithm will follow to construct valid polyketide structures.
- **Defining Production Rules:** The grammar consists of rules that specify how different elements, such as domains and molecules, can be combined to form a polyketide. These rules dictate the allowed combinations and order of components in the final polyketide.
- **Customization:** While PKevo provides a default grammar suitable for many tasks, you may need to customize it to match your specific objectives. This step allows you to fine-tune the production rules, ensuring that the generated polyketides align with your goals.
- **Phenotype Construction:** Keep in mind that in PKevo, a phenotype represents the synthesized polyketide. The grammar's production rules play a crucial role in constructing these phenotypes from the genetic information contained within individuals.

An example of such a phenotype could be: 

"AT(Malonyl)-ACP--KS-AT(Chlorethylmalonyl)-ACP--KS-AT(Hydroxymalonyl)-KR-ACP--KS-AT(Aminomalonyl)-KR-DH-ACP--TE(Macrolactonization)-Cyclization(3)"

In [3]:
GRAMMAR_FILE_NAME = 'pytest_pks.bnf'
current_dir = os.path.dirname(os.path.abspath('__file__'))
grammar_file_path = os.path.join(current_dir, '..', 'grammatical_evolution', 'grammars', GRAMMAR_FILE_NAME)

grammar = _Grammar(grammar_file_path=grammar_file_path) 


Setting up grammar...

Using grammar defined in /scratch/pkaleta/pkevo/pkevo/notebooks/../grammatical_evolution/grammars/pytest_pks.bnf

Loading grammar took (seconds): 0.004

Grammar setup complete.



In [4]:
# If you want to see how the grammar object looks like, execute this code block as well
print(grammar)

Grammar with following 25 non-terminals, 38 terminals and 25 rules.

Non-terminals:
<Termination_Reaction>, <KS_Domain>, <S>, <Loading_Module>, <ACP_Domain>, <PKS_Starter_Units>, <Termination_Module>, <ER_Domain>, <PKS_Extender_Unit>, <Elongation_Module>, <AT_Domain>, <PKS_Extender_Units>, <SAT_Domain>, <Number_Or_Any>, <PKS_Loading_Module>, <TE_Domain>, <PKS_Elongation_Domains>, <Termination_Reactions>, <Cyc_Spec>, <KR_Domain>, <Elongation_Modules>, <PKS_Beta_Processing_Domains>, <DH_Domain>, <PKS_Elongation_Module>, <PKS_Starter_Unit>

Terminals:
Butyryl, Chlorethylmalonyl, 2, -, Malonyl, AHBA, ACP, ), Ethylmalonyl, AT(, Isobutyryl, 1, 3-Methylbutyryl, Methoxymalonyl, Macrolactamization, Malonamyl, KR, Acetyl, Claisen, Macrolactonization, Benzoyl, Hydroxymalonyl, Propionyl, Coumaroyl, Hexanoyl, TE(, Hydrolysis, DH, ER, 2-Methylbutyryl, Methylmalonyl, Aminomalonyl, Cyclization(, KS, 3, Acetoacetyl, Cyclohexylcarbonyl, --

Rules:
  1. Rule: <S> -> [[('<Loading_Module>', 'NT'), ('--', '

### Step 3: Initializing the first generation with "random" individuals
**A Word of Precaution:** In the beginning we said that this tutorial aims to show you *how PKevo works* rather than *how you can use PKevo*. This distinction is crucial. The reason is that for this tutorial we will be diving deep into internal functions from packages and sub-packages inside PKevo directly to give you a better understanding of the underlying steps and processes involved. 

If you are mainly interested in how to use the tool for your simulations and don't care too much about how it internally works (an attitude I'm all too familiar with), please refer to the provided Wiki and README in the Git repository. 

In [5]:
# Let's create our 4 individuals for the initial generation manually
# Note: PKevo would generate those randomly and set the optimization type and default fitness based on the used fitness function
ind1 = _Individual(optimization_type='max', default_fitness=0.0, genome=[19, 11, 9, 4, 25, 27, 22, 10, 11, 16, 14, 29, 17, 21, 7, 5, 3, 6, 11, 22, 13, 8, 8, 4, 10, 16, 16, 14, 8, 23, 20, 27, 17, 21, 17, 25, 12, 25, 24, 2])
ind2 = _Individual(optimization_type='max', default_fitness=0.0, genome=[3, 5, 19, 2, 0, 23, 13, 23, 19, 27, 1, 2, 19, 15, 30, 11, 14, 5, 0, 24, 9, 23, 0, 5, 4, 4, 9, 29, 5, 7, 11, 10, 13, 3, 3, 18, 24, 28, 2, 2])
ind3 = _Individual(optimization_type='max', default_fitness=0.0, genome=[101, 125, 35, 93, 57, 127, 71, 81, 83, 110, 13, 27, 13, 38, 121, 112, 40, 27, 112, 88, 85, 20])
ind4 = _Individual(optimization_type='max', default_fitness=0.0, genome=[101, 35, 122, 127, 72, 83, 84, 79, 83, 88, 48, 79, 85, 121, 59, 112, 40, 27])
# Note: Those 4 individuals were chosen in such a way that (at least with the chemistry setup at the time of writing this tutorial):
#   - ind1 and ind2 produce multiple molecules
#   - ind3 produces zero molecules
#   - ind4 produces 4 molecules

# Let's add those 4 individuals to our initial generation (a generation is nothing else then a list of individuals)
generation_0 = [ind1, ind2, ind3, ind4]

# As you can see it currently has no phenotype, which we need to change, before we can add the individual to our initial generation
for ind in generation_0:
    # Besides the actual phenotype our grammar also gives us statistical data and the assembly instructions for the phenotype
    # as you can see, as input the grammar only requires the genome of the individual
    ind.phenotype, ind.used_codons, ind.assembly_line = grammar.generate_phenotype(ind.genome)

# Note: For the sake of a better understanding and easier identification, we will give our individuals different names. 
# In PKevo names serve no specific purpose and were only introduced for debugging purposes during development.
for idx, ind in enumerate(generation_0, start=1):
    ind.pet_name = 'initial_ind-'+str(idx)

In [6]:
# To get a better understanding of how an individual looks like right after creation, we will examine one a bit closer
print(ind1)

Individual 'initial_ind-1':
  Genotype:  [19, 11, 9, 4, 25, 27, 22, 10, 11, 16, 14, 29, 17, 21, 7, 5, 3, 6, 11, 22, 13, 8, 8, 4, 10, 16, 16, 14, 8, 23, 20, 27, 17, 21, 17, 25, 12, 25, 24, 2]
  Genotype length:  40
  Used codons:  25
  Phenotype: AT(3-Methylbutyryl)-ACP--KS-AT(Malonyl)-KR-DH-ACP--KS-AT(Hydroxymalonyl)-ACP--KS-AT(Aminomalonyl)-ACP--KS-AT(Aminomalonyl)-KR-DH-ACP--KS-AT(Methylmalonyl)-KR-DH-ER-ACP--KS-AT(Chlorethylmalonyl)-KR-DH-ER-ACP--TE(Macrolactamization)
  Generated molecules: 0
  Best molecule (SMILES): None
  Fitness:   0.0



### Step 4:  Initializing our Chemistry (Setting up the rules for polyketide creation)
In this step (and the next), we venture slightly away from Grammatical Evolution, focusing instead on preparing our polyketides for evaluation. While not directly related to Grammatical Evolution, this process is essential for PKevo to assess pharmacophore docking fitness accurately.

To achieve this, we must first convert our application-specific PKS-Domain strings into a more universally recognized format - chemical molecules represented as canonical SMILES. This transformation enables the third-party tool *LigandScout* to calculate the desired fitness scores using docking simulations.

The generation of these molecules is handled by another third-party tool called *MØD*. Only after generating molecules for each individual can we proceed with the final fitness evaluation.

**Note:** To modify the behavior of our chemistry, you can either edit existing files or add new ones to the `/chemistry_model` folder. However, for this tutorial, we will rely on the existing files. For more in-depth information on working with chemistry in PKevo, please consult the Wiki.


In [7]:
# To initialize our Chemistry we need:
#   1) Molecules (to start and extend the polyketide chain)
#   2) Reactions (to construct the polyketide)
#   3) (Sub-)strategies for polyketide construction by defining the 'game rules' of our chemistry
#   4) Mapping between PKS domains (more specifically the grammar production rules) and strategies

# This initializes our chemistry by using all available resources inside the `chemistry` package's `chemistry_model` folder.
pks_chem = Chemistry(verbose=False)

# Note: Now that the chemistry is set, we can continue with our GE algorithm. Visualizing the currently existing molecules, 
# reactions and substrategies in here is rather difficult, since they exist as objects provided by the external tool MØD and 
# don't offer a simple way of visualization inside the CLI.

### Step 5: Generating molecules from PKS-Domainstrings
In Step 3, we initiated our starting generation, named "Generation Zero." Each of the four individuals now possesses a genome and a phenotype. However, they lack a fitness score beyond their default score, set initially to `0.0`. Additionally, they lack any molecules - a necesserary requirement for pharmacophore fitness evaluation.

Thanks to our established chemistry (as discussed in Step 4), we can now, for each generated phenotype (or, more precisely, for their respective phenotype assembly instructions) construct a dedicated strategy. This strategy is used by *MØD* to generate the released product molecules for a specific PKS Domain String of an individual.


In [8]:
# Iterate over all individuals in our generation 0 and create the released product molecules
for ind in generation_0:
    # Note: We can only generate molecules if the individual has an expressed phenotype (and therefore an assembly guide for that phenotype)
    # For this tutorial we chose our individuals specifically so that this will be the case, however it is good practise to check nonetheless
    if ind.assembly_line:
        # Note: The molecules list of an individuals contains objects of type `Molecule`, which is a dataclass specifically created for PKevo
        # Another note: PKevo uses multiprocessing here to improve performance, however for just 4 individuals we can afford to call MØD sequentially
        ind.molecules = [Molecule(smiles_string, 0.0) for smiles_string in pks_chem.get_smiles_for_ind(ind.assembly_line)]

In [9]:
# Let's have a look at our freshly generated `Molecules`
for ind in generation_0:
    print(ind.molecules)
    print("=============================================")

[Molecule(_smiles_string='C1(C(CCCl)CC(C)CC(N1)=CC(C(C(C(C=CCC(C)C)=O)O)=O)N)=O', fitness=0.0), Molecule(_smiles_string='C(C(C(C=CCC(C)C)=O)O)(C1C=C(CC(C)CC(C(N1)=O)CCCl)N)=O', fitness=0.0)]
[Molecule(_smiles_string='C1(C(C(CC(C(CC)=CC(CC)=CC(CC)C(C)O)=O)O1)N)=O', fitness=0.0), Molecule(_smiles_string='C1(C(C(CC(C(CC)=CC(CC)=CC(CC)C(C)O1)=O)O)N)=O', fitness=0.0)]
[Molecule(_smiles_string='C(C(C(C(CCC=C(C(O)=O)C)CC)O)O)(O)=O', fitness=0.0)]
[Molecule(_smiles_string='C(C1(C(C(C=C(C1=O)N)CCCl)=O)O)(O)=O', fitness=0.0), Molecule(_smiles_string='C(C1(C(C(C=C(C1=O)N)CCCl)=O)O)(O)=O', fitness=0.0)]


### Step 6: Evaluating the fitness of our individuals
At last, we arrived at the point we've all been waiting for - it's time to evaluate our individuals. For that, we will return to the Grammatical Evolution aspect of PKevo, but let's just for a quick moment squeeze in a bit more theory before actually getting to the fitness evaluation itself.

A fundamental component of all evolutionary algorithms is the fitness evaluation. It plays a crucial role in helping the algorithm determine which individuals advance to the next generation and which individuals will fade into obscurity. Typically, a fitness function can be designed to evaluate each individual independently, without considering the other individuals in the generation, or it can take a collective approach, assessing the entire generation together (a concept known as co-evolution). PKevo accommodates both types of fitness evaluation. In the case of the pharmacophore fitness function, it evaluates the entire generation, although not for reasons of co-evolution.

The decision to evaluate the entire generation when using the pharmacophore fitness function is primarily based on performance and how *LigandScout* operates. However, these technical details are beyond the scope of this tutorial, so we won't delve into them here. Just keep in mind that the effectiveness of molecule X's docking to the target pharmacophore is in no way influenced by the presence of molecule Y in the same generation. Consequently, we can't consider this fitness function as co-evolutionary, even though the entire generation is evaluated together.

With that being said, let's move on to the actual evaluation process...

In [10]:
# Note: This part of the workflow takes by far the longest amount of time. Depending on your system, 
# it may take a couple of minutes to finish. 

# Load the pharmacophore fitness function
pharma_fitness = FitnessFunctions.PharmacophoreScore()
# This cache allows to speed things up for upcoming generations. Only 'unknown' molecules will be send to LigandScout for evaluation
cache_dict = dict()
# Call the fitness function to evaluate the fitness of each individual
pharma_fitness.objective_function(individuals=generation_0, remembered_pharma_scores=cache_dict)

# At this point each individual has now an actual fitness score and the first iteration of the GE simulation is almost complete
# We must now sort the list of individuals based on their fitness scores in the correct order from best to worst
generation_0.sort(reverse=True)
# We must also store our best individual separately, since depending on the chosen evolutionary configuration
best_individual_ever = generation_0[0]

In [11]:
# Up until now each individual only had its pre-defined default fitness. Now let's assess their true fitness scores
print("Achieved fitness scores for individuals of Generation 0")
for ind in generation_0:
    print(f"    Fitness: {ind.fitness}")

Achieved fitness scores for individuals of Generation 0
    Fitness: 0.48400527
    Fitness: 0.0
    Fitness: 0.0
    Fitness: 0.0


### Interim Conclusions
We did it! It's done! Well almost... or actually not at all. We are nowhere near the finish line yet unfortunately. If you have ever heard about evolutionary algorithms, you might be wondering at which point all the mutations and crossovers on the genomes occured and when exactly we selected the parent individuals for creating offspring. It was not mentioned, because it hasn't happened yet. The initial generation is special in that regard, which is why we will continue the evolutionary loop for one more iteration, in order to show those aspects of evolutionary algorithms as well.

Also, it is important to note that evolutionary algorithms will rarely (if ever) finish this fast with so few individuals. The fact that one of our four individuals was able to score a fitness value of above 0.0 is not a coincidence, but rather due to careful planning and selection when we first initialized our generation 0. For the tutorial this few individuals and just two round of generations will be enough to demonstrate how PKevo works, but for real usage you will need a lot more than that. But those are things you can dive deeper into by reading the Wiki and the README. 

### Step 7: Selecting parents ("Starting the next generation")
As you will see with various other aspects of evolutionary algorithms, there are usually multiple ways of how to achieve your task. Same goes for this step. For selecting parents we will be using the 'tournament-style' selection method, which will randomly draw a number of individuals from the current generation and add the one with the best fitness to the "pool of potential parents". This is done until this pool reaches the size of the generations (so in our case we will have 4 potential parents).

As you can probably guess, having a better fitness increases your chance of making it into the pool, however there is no guarantee that the best individual will always make it. Also, it is quite possible that certain individuals will be represented in this pool multiple times. This demonstrates that one parent can reproduce multiple times during a generation.

**Note:** If you're afraid that potentially bad parents made the cut, feel free to run the next bit of code again and again until you're satisfied.

In [29]:
# Now let's select the possible parents for the next generation using the tournament-style selection method    
parents = tournament_selection(generation_0)

In [30]:
# Let's print the names of our current indviduals and their respective fitness score
print("Individuals of Generation 0:")
for ind in generation_0:
    print(f"    Name: {ind.pet_name}, Fitness score: {ind.fitness}")

# Let's check who made the cut
print("\nPool of potential parents for Generation 1:")
for ind in parents:
    print(f"    Name: {ind.pet_name}, Fitness score: {ind.fitness}")

Individuals of Generation 0:
    Name: initial_ind-3, Fitness score: 0.48400527
    Name: initial_ind-1, Fitness score: 0.0
    Name: initial_ind-2, Fitness score: 0.0
    Name: initial_ind-4, Fitness score: 0.0

Pool of potential parents for Generation 1:
    Name: initial_ind-3, Fitness score: 0.48400527
    Name: initial_ind-1, Fitness score: 0.0
    Name: initial_ind-1, Fitness score: 0.0
    Name: initial_ind-3, Fitness score: 0.48400527


### Step 8: Performing the Crossover ritual
This should be self-explanatory, right? Wrong! So here we go...

This step is all about generating offspring from the potential parent candidates. There are several different crossover functions and PKevo enables you to perform onepoint- and twopoint-crossovers with either variable- or fixed-length genomes and with the option to split genomes only in the coding regions of the genome or all across the whole genome. You guessed it, for any further details please refer to the Wiki. Here in this tutorial we will go with the simple option: Singlepoint-crossover, which will occur only in the coding regions of the genomes and we will keep the original lengths of the genomes.

#### About the selected operation and choosen options
**Singlepoint-crossover operation:** On the genome we select just one point, at which the genome will be split. Then we take of the two parts of the genome and combine it with one of the two parts from the other genome. The remaining two parts from both genomes are also combined together. For example:

**within-coding-reagion option:** When generating the phenotype from a genome, the algorithm starts from the left most position in the genome array (`1..n`) and works its way further right until it reaches the end of the genome or until it runs out of production rules, in which case we will have a valid phenotype generated by the codons `1` to `k` on the genome, while codons `k=1` to `n` did no contribute to that particular phenotype. The described option allows us to prohibit crossovers in the tail of the genome, which in most cases leads to no changes occuring in the resulting phenotype.

**fixed-lenght option:** With this enabled, the index of the crossover point will be the same on both parental genomes, ensuring that the resulting genomes also retain the same lengths. Otherwise crossover point `a` on the first genome might be further in the beginning than crossover point `b` on the second genome, which results in the offspring genomes to grow and shrink in size.

**Singlepoint crossover with fixed-length example**:

Parent 1: xxxxxxx|xxx (genome lenght = 10) -> Offspring A: xxxxxxxyyy (genome lenght = 10)

Parent 2: yyyyyyy|yyy (genome lenght = 10) -> Offspring B: yyyyyyyxxx (genome lenght = 10)


In [37]:
# Initialize list of our offsprings
offsprings = []

# Create offspring (by randomly chosing two parents) until we reached desired generation size (in our case this is 4)
while len(offsprings) < 4:
    offsprings.extend(onepoint_crossover(*random.sample(parents, 2), default_fitness=0.0, within_used=True, variable_len=False))

# Note: we also want to make sure that our offspring gets a phenotype as well
for idx, child in enumerate(offsprings, start=1):
    child.phenotype, child.used_codons, child.assembly_line = grammar.generate_phenotype(child.genome)
    child.pet_name = 'child-'+str(idx)

In [38]:
# Let's analyze the genomes of the selected (potential) parents and compare them with the generated genomes of the children
for parent in parents:
    print(f"Parent genome: {parent.genome}")
print()
for child in offsprings:
    print(f"Child genome: {child.genome}")

Parent genome: [101, 125, 35, 93, 57, 127, 71, 81, 83, 110, 13, 27, 13, 38, 121, 112, 40, 27, 112, 88, 85, 20]
Parent genome: [19, 11, 9, 4, 25, 27, 22, 10, 11, 16, 14, 29, 17, 21, 7, 5, 3, 6, 11, 22, 13, 8, 8, 4, 10, 16, 16, 14, 8, 23, 20, 27, 17, 21, 17, 25, 12, 25, 24, 2]
Parent genome: [19, 11, 9, 4, 25, 27, 22, 10, 11, 16, 14, 29, 17, 21, 7, 5, 3, 6, 11, 22, 13, 8, 8, 4, 10, 16, 16, 14, 8, 23, 20, 27, 17, 21, 17, 25, 12, 25, 24, 2]
Parent genome: [101, 125, 35, 93, 57, 127, 71, 81, 83, 110, 13, 27, 13, 38, 121, 112, 40, 27, 112, 88, 85, 20]

Child genome: [101, 125, 35, 93, 57, 127, 71, 81, 83, 110, 13, 27, 13, 21, 7, 5, 3, 6, 11, 22, 13, 8, 8, 4, 10, 16, 16, 14, 8, 23, 20, 27, 17, 21, 17, 25, 12, 25, 24, 2]
Child genome: [19, 11, 9, 4, 25, 27, 22, 10, 11, 16, 14, 29, 17, 38, 121, 112, 40, 27, 112, 88, 85, 20]
Child genome: [101, 11, 9, 4, 25, 27, 22, 10, 11, 16, 14, 29, 17, 21, 7, 5, 3, 6, 11, 22, 13, 8, 8, 4, 10, 16, 16, 14, 8, 23, 20, 27, 17, 21, 17, 25, 12, 25, 24, 2]
Child ge

### Step 9: Mixing in some mutations
We have our offspring for the next generation, but they are not fully ready yet. With the help of the crossover operation we managed to introduce some change into the genomes (and change is a driving force behind evolution), but that is not enough. Values that are not present in the parental genome pool will never occur in any of the generation if we rely on crossover alone. We need to introduce additional mutations to the genome, to achieve a satisfying amount of change for upcoming generations.

**Note:** PKevo performs a couple of different mutations, but we will here only focus on the single point mutations of individuals codons.

In [39]:
# For each child we will go over the genome and randomly introduce changes to the individuals codons
for child in offsprings:
    # Just like with crossover, we only want to perform such changes inside the coding region of the genome
    child = mutations.int_flip_mutation(child, within_used=True)

In [40]:
# And this is how the genome has mutated
for child in offsprings:
    print(f"Child genome (mutated): {child.genome}")

Child genome (mutated): [101, 125, 35, 93, 57, 127, 71, 81, 83, 13, 13, 27, 13, 21, 7, 5, 3, 6, 11, 22, 13, 8, 8, 4, 10, 16, 16, 14, 8, 23, 20, 27, 17, 21, 17, 25, 12, 25, 24, 2]
Child genome (mutated): [19, 11, 22, 4, 25, 27, 22, 10, 11, 16, 14, 29, 17, 38, 121, 112, 19, 27, 112, 88, 85, 20]
Child genome (mutated): [101, 11, 9, 4, 25, 5, 22, 26, 11, 16, 14, 29, 12, 7, 7, 5, 3, 6, 11, 14, 13, 8, 8, 4, 19, 16, 16, 14, 8, 23, 20, 27, 17, 21, 17, 25, 12, 25, 24, 2]
Child genome (mutated): [19, 125, 35, 6, 57, 127, 10, 81, 83, 110, 13, 27, 21, 38, 121, 112, 40, 27, 112, 88, 85, 20]


### Step 10: Generating molecules and evaluating the offspring
Before we can compare the existing individuals from generation zero with our new, hopefully fitter, offspring, we will again need to first generate the necessary molecules and afterwards evaluate the fitness of each child. Once this is achieved, we can make the final selection for the new generation from the existing individuals. 

**Note:** The molecule generation and fitness evaluation of the offspring follow the same logic as we already showed in steps 5 and 6.

In [41]:
# Note: This part of the workflow takes by far the longest amount of time. Depending on your system, 
# it may take a couple of minutes to finish. 

# Iterate over all children and generate the released molecules
for child in offsprings:
    if child.assembly_line:
        child.molecules = [Molecule(smiles_string, 0.0) for smiles_string in pks_chem.get_smiles_for_ind(child.assembly_line)]

# Call the fitness function again, but this time we want to evaluate the offspring
pharma_fitness.objective_function(individuals=offsprings, remembered_pharma_scores=cache_dict)

### Step 11: Replacing individuals in upcoming generation (*"How Generation 1 is born"*)
To recapitulate, we have 4 individuals from our "Generation 0" and we have 4 offspring individuals originating from individuals of "Gen Zero". The goal is to create the next generation, "Generation 1", which again should contain 4 individuals. So, what do we do? The answer to that lies in the *Replacement strategy*. There are plenty of those out there and we won't be getting into any details, but what they all have in common, is that they consider the fitness score of an individual when making a decision. 

PKevo implements 2 different replacement strategies:
 - *Generational replacement* strategy (which we will be using in this tutorial), where the *x* best individuals from the previous generation are considered for moving up into the next generation
 - *Steady-state replacement* strategy, where the best individual from the new population automatically makes it into the next generation (basically meaning that the new generation is just the old generation expanded by the best offspring individual). 

In [42]:
# Using the generation replacement strategy, let's fill our next generation with individuals
generation_1 = generational_replacement(offsprings, generation_0)

In [43]:
# Let's take a look at what we have to choose from (which is our Gen Zero and the offspring individuals)
print("Achieved fitness scores for individuals of 'Generation 0':")
for ind in generation_0:
    print(f"    Name: {ind.pet_name}, Fitness: {ind.fitness}")

print("\nAchieved fitness scores for offspring individuals:")
for child in offsprings:
    if 'child' in child.pet_name:
        print(f"    Name: {child.pet_name}, Fitness: {child.fitness}")

# And these are our individuals that were selected for the new generation
print("\nThese are the individuals of 'Generation 1':")
for ind in generation_1:
    print(f"    Name: {ind.pet_name}, Fitness: {ind.fitness}")

Achieved fitness scores for individuals of 'Generation 0':
    Name: initial_ind-3, Fitness: 0.48400527
    Name: initial_ind-1, Fitness: 0.0
    Name: initial_ind-2, Fitness: 0.0
    Name: initial_ind-4, Fitness: 0.0

Achieved fitness scores for offspring individuals:
    Name: child-1, Fitness: 0.0
    Name: child-2, Fitness: 0.0
    Name: child-3, Fitness: 0.0
    Name: child-4, Fitness: 0.0

These are the individuals of 'Generation 1':
    Name: initial_ind-3, Fitness: 0.48400527
    Name: child-1, Fitness: 0.0
    Name: child-2, Fitness: 0.0
    Name: child-3, Fitness: 0.0


## Conclusion
You have now witnesses something truly beautiful - the birth of a new generation. With that, this tutorial comes to an end, however an actual simulation perfomed by PKevo would most likely continue for many more generations and generate vastly more individuals in the process. 

Something you might have also noticed is the involvement of random chance in an evolutionary algorithm like Grammatical Evolution. Executing the code in this tutorial can therefore lead to different results. Our initial generation was 'hard-coded' and will therefore remain the same each time (something which in a 'real' PKevo simulation would've also been left to the God of randomness), but already in our parent selection, we might run into cases where this tutorial will show you that only a single individual was chosen four times, or (possibly even worse) the pool of potential parents will only contain our individuals that received a fitness score of 0.0, completely ignoring our 'best-performing' individual. 

As you can probably imagine, such drastic cases can lead to drastic results in the simulations, which is why it is generally advised to use larger populations in your generations and repeat the evolutionary loop over multiple generations, in order for the algorithm to assure that you are covering a large enough portion of the solution space for the given problem at hand (in the case of PKevo the generation of polyketides that can dock to a given pharmacophore).

### What's next?
Well, you should hopefully by now have a better understanding of the internal workflows of PKevo and you might have even learned a thing or two about evolutionary algorithms in general. If you managed to execute all of the code in this tutorial, it is a good indication that you are able to execute PKevo on its own. So, there is nothing stopping you from running one or two simulations on your own now. Although, it is probably a good idea to check out the Wiki or the README to get additional information about how you can use PKevo for your simulations. 

Another source for getting further insight into PKevo (especially if you're interested in the code) are also the provided test classes (which you can find inside the `/tests` folder) and the source code itself obviously.